In [12]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy.ext.declarative import declarative_base


engine = create_engine('sqlite:///:memory:') # Create an SQLite database in memory (replace with your desired database URL)


Base = declarative_base() # Create a base class for declarative models

# Defining the Student class representing the 'student' table
class Student(Base):
    __tablename__ = 'student'

    id = Column(Integer, primary_key=True)
    name = Column(String(50), nullable=False)
    age = Column(Integer, nullable=False)

# Defining the Subject class representing the 'subject' table
class Subject(Base):
    __tablename__ = 'subject'

    id = Column(Integer, primary_key=True)
    name = Column(String(50), nullable=False)


# Defining the association table between students and subjects
class StudentSubject(Base):
    __tablename__ = 'student_subject'

    id = Column(Integer, primary_key=True)
    student_id = Column(Integer, ForeignKey('student.id'), nullable=False)
    subject_id = Column(Integer, ForeignKey('subject.id'), nullable=False)

   # Making relationships between tables
    student = relationship('Student', back_populates='subjects')
    subject = relationship('Subject', back_populates='students')

# Create many-to-many relationships
Student.subjects = relationship('StudentSubject', back_populates='student')
Subject.students = relationship('StudentSubject', back_populates='subject')

# Create the tables in the db
Base.metadata.create_all(engine)

# Create a session
Session = sessionmaker(bind=engine)
session = Session()

# Insert data into the tables 

# Insert data into the "student" table
student_data = [
    Student(name='Bae', age=18),
    Student(name='Eddy', age=21),
    Student(name='Lily', age=22),
    Student(name='Jenny', age=19)
]
session.add_all(student_data)

# Insert data into the "subject" table
subject_data = [
    Subject(name='English'),
    Subject(name='Math'),
    Subject(name='Spanish'),
    Subject(name='Ukrainian')
]
session.add_all(subject_data)

# Commit the changes to the session
session.commit()

# Insert data into the "student_subject" table
student_subject_data = [
    StudentSubject(student_id=1, subject_id=1),
    StudentSubject(student_id=2, subject_id=2),
    StudentSubject(student_id=3, subject_id=3),
    StudentSubject(student_id=4, subject_id=4),
    StudentSubject(student_id=1, subject_id=3),
    StudentSubject(student_id=3, subject_id=1)
]
session.add_all(student_subject_data)

# Commit the changes to the session
session.commit()

# Making Query of students who visited 'English' classes
english_students = (
    session.query(Student)
    .join(StudentSubject, Student.id == StudentSubject.student_id)
    .join(Subject, StudentSubject.subject_id == Subject.id)
    .filter(Subject.name == 'English')
    .all()
)

# Print the names of students who visited 'English' classes
for student in english_students:
    print(f'Student that visisted English classes: {student.name}')


Student that visisted English classes: Bae
Student that visisted English classes: Lily
